In [62]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, LSTM
from keras.layers import TimeDistributed
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import numpy as np
import itertools
import glob
import os
import json
import sys
import time

In [63]:
def get_model():
    model = Sequential([
        TimeDistributed(Conv1D(16,3, activation='relu', padding = "same"),input_shape=[10,33,2]),
        TimeDistributed(BatchNormalization()),
        #TimeDistributed(MaxPooling1D()),
        TimeDistributed(Dropout(0.5)),
        #TimeDistributed(Conv1D(64,3, activation='relu',padding = "same")),
        BatchNormalization(),
        #TimeDistributed(Dropout(0.8)),
        TimeDistributed(Flatten()),
        #TimeDistributed(Dense(30,activation='softmax')),  
        LSTM(20,unit_forget_bias = 0.5, return_sequences = True),
        TimeDistributed(Dense(6,activation='softmax'))        
    ])
    adam = Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])
    return model

In [64]:
asanas = {0:'bhujangasan', 1:'padamasan', 2:'shavasan', 3:'tadasan', 4:'trikonasan', 5:'vrikshasan'}

model = get_model()
model.load_weights("weights/val1-100-0.9945.hdf5", by_name=True)


In [65]:
def addToPred(Xch,count,Xarr,Xlist,seq):
    Xarr[count] = Xch
    count = count + 1
    #print(count)
    if count == 10:
        #print('Hello2')
        Xtemp = []
        Xtemp.append(Xarr)
        Xlist[seq] = Xarr
        Xarr = np.empty((10,33,2))
        count = 0
        seq  = seq + 1
        #printing prediction for first frame        
        Yht_pred = model.predict(np.asarray(Xtemp))
        Y_pred = np.argmax(Yht_pred[0],axis = 1)
        Y_pred_asanas = np.array([asanas[a] for a in Y_pred])
        
        print("Prediction " + " is " + str(Y_pred_asanas[0]))
        
    #print(count)
    return seq, count , Xarr

In [66]:
def makePred(filename,count,Xarr,Xlist,seq):
    with open(filename) as json_data:
        d = json.load(json_data)
        for i in range(0, len(d)):
            A = []
            A = d[i]
            #A = [person['pose_keypoints_2d']]                
            X = np.asarray(A)
            X1 =  X[:,0]
            X2 = X[:,1]
            Xch = np.dstack((X1,X2))
            seq ,count, Xarr = addToPred(Xch,count,Xarr,Xlist,seq)
            json_data.close
    return seq, count, Xarr

In [67]:
count = 0
seq = 0
Xarr = np.empty((10,33,2))
Xlist = np.empty((1000,10,33,2))
#print(Xarr.shape)
lastRead = "none"
timeout = time.time() + 1
latest_file = None  # Define the variable latest_file

# while(True):
#     #print(count)
#     list_of_files = glob.glob("output/5_Dristi_Shav.json")
#     if list_of_files:  # Check if the list of files is not empty
#         latest_file = max(list_of_files, key=os.path.getctime)
#         if lastRead != latest_file:
#             try:
#                 seq, count, Xarr = makePred(latest_file,count,Xarr,Xlist,seq)
#                 timeout = time.time() + 5 # will give result if no new file for 5 secs
#             except:
#                 print ("Frame issue")
#     if time.time() > timeout:
#         break
#     lastRead = latest_file
while(True):
    #print(count)
    latest_file = max(glob.glob("output/*"), key=os.path.getctime)
    seq, Xarr, count = makePred(latest_file,count,Xarr,Xlist,seq)      
    if time.time() > timeout:
        break

1/1 [==============================] - 1s 1s/step
Prediction  is shavasan
1/1 [==============================] - 0s 33ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 38ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 117ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 30ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 29ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 29ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 37ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 27ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 23ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 21ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 25ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 24ms/step
Prediction  is shavasan
1/1 [========

In [68]:
import os

# ...

count = 0
seq = 0
Xarr = np.empty((10,33,2))
Xlist = np.empty((1000,10,33,2))
#print(Xarr.shape)
lastRead = "none"
timeout = time.time() + 1
latest_file = None  # Define the variable latest_file

while(True):
    #print(count)
    try:
        list_of_files = glob.glob("output/*")
    except FileNotFoundError:
        # Handle the exception when the directory "output" does not exist
        print("Directory 'output' does not exist.")
        break
    
    if list_of_files:  # Check if the list of files is not empty
        latest_file = max(list_of_files, key=os.path.getctime)
        if lastRead != latest_file:
            try:
                seq, count, Xarr = makePred(latest_file,count,Xarr,Xlist,seq)
                timeout = time.time() + 5 # will give result if no new file for 5 secs
            except:
                print ("Frame issue")
    if time.time() > timeout:
        break
    lastRead = latest_file


1/1 [==============================] - 0s 27ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 31ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 24ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 26ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 32ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 39ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 33ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 30ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 33ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 30ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 32ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 33ms/step
Prediction  is shavasan
1/1 [==============================] - 0s 29ms/step
Prediction  is shavasan
1/1 [=======